# Tic-tac-toe minmax algorithm
We will demonstrate the minmax algorithm on a game of Tic-tac-toe played on a 3x3 game board.

In [1]:
import numpy as np

The State class captures the current state of the game.
* **Attributes**
    * gameplan - 3x3 game board with values 0 to 2
    * player - the player who is currently on the turn
    * current_player - the player will analyze the game and keep track of possible new states. Players take turns, so the new state should be viewed from the perspective of the opposing player. the player who is on the turn in the current state when searching the state space
    * depth - the depth of the analyzed state
    * max_depth - how many moves ahead the maximum is looking. If depth = max_depth, I don't analyze the game any further.

* **Methods**
    * terminal_test - method returns information whether the current state is final or not. If it is, it returns the winner.
    * utility - the method tries to evaluate the current state from the player's perspective. In the basic version, it only distinguishes whether the player wins, doesn't win or the move doesn't lead to the end of the game.
    * possible_actions - the method returns a list of possible moves. In the case of biscuits, this will be the coordinates of the playing area where the playing stone can be placed.
    * expand - this method takes the current state and action definition (the coordinates where to place the die) and creates a new game state.
    * minmax - custom implementation of the minmax algorithm
    * next_current_player - returns the opponent to the current_player variable
    * next_player - returns the opponents to the player variable    

In [2]:
class State:
    """ Capturing the state of the game
    gameplan - two-dimensional 3x3 array
             - 0 - empty array
             - 1 - X
             - 2 - O
    player - the player who has the turn in the game

    current_player - player who is on the turn in the given state when searching the state space
    depth - depth of the state space search
    max_depth - maximum length of the search
    """

    generated = 0

    def __init__(self, gameplan, player, current_player=None, depth=0, max_depth=3):
        self.gameplan = gameplan
        self.player = player
        if current_player is None:
            self.current_player = player
        else:
            self.current_player = current_player
        self.depth = depth
        self.max_depth = max_depth
        
        State.generated += 1

    def terminal_test(self):
        """ The method tests the current state and returns a value indicating whether the game is finished and, if so, who the winner is

            0 - no final status
            1 - 1 wins
            2 - winner 2
        """

        # end of game rules - horizontal and vertical triplets
        for i in range(3):
            if np.array_equal(self.gameplan[i], [1, 1, 1]):
                return 1
            if np.array_equal(self.gameplan[i], [2, 2, 2]):
                return 2
            if np.array_equal(self.gameplan[:, i], [1, 1, 1]):
                return 1
            if np.array_equal(self.gameplan[:, i], [2, 2, 2]):
                return 2

        # end of game rules - diagonals
        if np.array_equal(self.gameplan.diagonal(), [1, 1, 1]):
            return 1
        if np.array_equal(self.gameplan.diagonal(), [2, 2, 2]):
            return 2
        if np.array_equal(np.fliplr(self.gameplan).diagonal(), [1, 1, 1]):
            return 1
        if np.array_equal(np.fliplr(self.gameplan).diagonal(), [2, 2, 2]):
            return 2

        return 0

    def utility(self, result):
        """ The method returns an evaluation of the current state of the game
            0 - draw
            1 - the player who is on the turn wins
            -1 - the opposing player wins
        """
        if result == 0:
            return 0
        elif result == self.player:
            return 1
        else:
            return -1

    def possible_actions(self):
        """ The method returns a list of possible actions for the given state
            The action is expressed by the coordinates of an empty playing field.
        """
        possible_actions = []
        # Finding empty playing fields
        for i in range(3):
            for j in range(3):
                if self.gameplan[i][j] == 0:
                    possible_actions.append((i, j))
        return possible_actions

    def expand(self, select_action):
        """ The method creates a new game state by applying the selected action to the current state

            In the new state, the opposing player will have the turn, but the state will be evaluated from the original player's perspective
            The depth of the searched state space will also increase
        """
        if select_action[0] not in range(3):
            return None
        if select_action[1] not in range(3):
            return None

        # playing field must be clear
        if self.gameplan[select_action[0], select_action[1]] != 0:
            return None
        
        new_array = np.copy(self.gameplan)
        new_array[select_action[0], select_action[1]] = self.current_player
        return State(new_array, 
                     self.player, 
                     self.next_current_player(), 
                     self.depth + 1, 
                     max_depth=self.max_depth)
        

    def minmax(self, strategy="max"):
        """"
        The method selects the one that matches the strategy from the possible actions for the given game state

        stategy - what strategy will be used to select from the possible actions
        """
        
        # checking the state of the game, for a completed game the rating from the utility method is returned
        result = self.terminal_test()
        if result != 0:            
            return self.utility(result), action
        

        # initialization of values for each strategy
        if strategy == "max":
            selected_utilization_value = float('-inf')
            next_strategy = "min"
        else:
            selected_utilization_value = float('inf')
            next_strategy = "max"

        # finding possible moves
        actions = self.possible_actions()

        # the selected action is filled with the first action of the possible actions
        selected_action = actions[0]

        # finding the optimum action from all possible acttions
        for action in actions:

            # state expansion
            expanded_state = self.expand(action)

            #  check the game ending for expanded state
            result = expanded_state.terminal_test()

            if result != 0:
                # games end, return status and action rating
                return expanded_state.utility(result), action
            else:
                # game continues
                # if there is at least one state in the expanded state, continue to expand the state, otherwise it's a draw
                if len(expanded_state.possible_actions()) == 0:
                    utilization = 0
                else:
                    # recursive call
                    utilization, _ = expanded_state.minmax(next_strategy)

                # according to the strategy, the evaluated action is chosen as the selected action
                if utilization > selected_utilization_value and strategy == "max":
                    selected_utilization_value = utilization
                    selected_action = action
                elif utilization < selected_utilization_value and strategy == "min":
                    selected_utilization_value = utilization
                    selected_action = action

        return selected_utilization_value, selected_action

    def next_current_player(self):
        """ The method returns the opponent's for state space searching
        """
        return 3 - self.current_player

    def next_player(self):
        """ Method returns opponent
        """
        return 3 - self.player

# Game of tic-tac-toe

Creating the initial state of the batch
* Game plan is empty (0)
* Game 1 is on the turn

In [3]:
state = State(gameplan=np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]]),
              player=1, max_depth=3)

Printing the initial state

In [4]:
print(state.gameplan)

[[0 0 0]
 [0 0 0]
 [0 0 0]]


A cycle of a game played against each other by two copies of the identical minmax algorithm
* The game runs in a loop that ends when one of the players wins or end with drawn.
* If the game has not ended the player on the turn using the minmax algorithm gets the turn to play
* The player takes an action and creates a new game state
* After the printing current state of game, the player passes the game to the opposing player.


Keep track of time and number of states generated as the game progresses. As the game progresses, the algorithm searches a smaller state space and the turn time decreases.

Since there is no constraint in the algorithm, both players play optimally. Therefore, the game ends in a draw.

In [5]:
while True:
    # Check if the game is not over
    game_result = state.terminal_test()
    if game_result != 0:
        print(f"Winner is {game_result} ")
        break

    # Checking for drawn
    if len(state.possible_actions()) == 0:
        print("Drawn")
        break

    # player move
    print(f"=====================\nPlayer {state.player}")
    _, player_action = state.minmax("max")
    print(f"Select action: {player_action}")
    state = state.expand(player_action)
    print(state.gameplan)
    print(f"Generated states {State.generated}.")
    State.generated = 0

    # switching the game to the other player
    state.player = state.next_player()

Player 1
Select action: (0, 0)
[[1 0 0]
 [0 0 0]
 [0 0 0]]
Generated states 269175.
Player 2
Select action: (1, 1)
[[1 0 0]
 [0 2 0]
 [0 0 0]]
Generated states 26853.
Player 1
Select action: (0, 1)
[[1 1 0]
 [0 2 0]
 [0 0 0]]
Generated states 2425.
Player 2
Select action: (0, 2)
[[1 1 2]
 [0 2 0]
 [0 0 0]]
Generated states 77.
Player 1
Select action: (2, 0)
[[1 1 2]
 [0 2 0]
 [1 0 0]]
Generated states 66.
Player 2
Select action: (1, 0)
[[1 1 2]
 [2 2 0]
 [1 0 0]]
Generated states 17.
Player 1
Select action: (1, 2)
[[1 1 2]
 [2 2 1]
 [1 0 0]]
Generated states 10.
Player 2
Select action: (2, 1)
[[1 1 2]
 [2 2 1]
 [1 2 0]]
Generated states 5.
Player 1
Select action: (2, 2)
[[1 1 2]
 [2 2 1]
 [1 2 1]]
Generated states 2.
Drawn
